## Task: GitHub Issue Tagger

### Overview

The GitHub Issue Tagger automates the process of fetching open GitHub issues, generating concise summaries for each, classifying them against available repository labels, and automatically applying those labels back to the issues.

In [25]:
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

Creating `Julep` Client

In [26]:
from julep import Client
import os

JULEP_API_KEY = "INSERT YOUR JULEP API KEY HERE"

# Create a Julep client
client = Client(api_key=JULEP_API_KEY, environment="dev")

Creating an `Secret` (Optional)

In [6]:
secret = client.secrets.create(
    name="github_api_key",
    value="INSERT YOUR GITHUB TOKEN HERE",
)

print(f"Created secret: {secret.name}")

Created secret: github_api_key


Creating an `agent`

In [27]:
# Defining the agent
name = "Robert"
about = "A github assistant."

# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="gpt-4o",
)

Defining a `Task`

In [28]:
import yaml


task_yaml = '''
name: "GitHub Issue Tagger"
description: "Fetch open issues via the GitHub API, summarize each one, classify for labels, and push those labels back."

input_schema:
  type: object
  properties:
    repo_owner:
      type: string
    repo_name:
      type: string
    max_issues:
      type: integer
      default: 20
  required: ["repo_owner", "repo_name"]

tools:
  - name: github_api
    type: api_call
    api_call:
      method: GET
      url: "https://api.github.com/"      # Placeholder, overridden in each subworkflow
      headers:
        Authorization: $ f"Bearer {secrets.github_api_key}" # Insert your github token here or use a secret


# -------------------- fetch_issues --------------------
fetch_issues:
  - tool: github_api
    arguments:
      method: GET
      url: $ f"https://api.github.com/repos/{steps[0].input.repo_owner}/{steps[0].input.repo_name}/issues?state=open&per_page={steps[0].input.max_issues}"
  - evaluate:
      issues: |-
        $ [
          {"number": i["number"], "title": i["title"], "body": i["body"]}
          for i in steps[0].output.json
        ]


# -------------------- fetch_labels --------------------
fetch_labels:
  - tool: github_api
    arguments:
      method: GET
      url: $ f"https://api.github.com/repos/{steps[0].input.repo_owner}/{steps[0].input.repo_name}/labels"
  
  - evaluate:
      all_labels: $ [label.name for label in steps[0].output.json]
  

# ------------------ summarize_issue -------------------
summarize_issue:
  - prompt:
    - role: user
      content: |-
        $ f"""
        Issue #{steps[0].input.issue.number}: {steps[0].input.issue.title}

        {steps[0].input.issue.body}

        Write a concise (1-3 sentence) summary of this issue.
        """
    unwrap: true

# ------------------ classify_issue ------------------
classify_issue:
  - prompt:
    - role: user
      content: |- 
        $ f"""
        Given this issue summary, suggest relevant GitHub labels as a JSON array.
        Only output JSON, e.g. ["bug","documentation"].

        Summary: {steps[0].input.summary}
        Available labels: {steps[0].input.all_labels}
        Note that you can only use available labels.
        """
    unwrap: true
  - evaluate:
      labels: $ load_json(steps[0].output.replace("```json", "").replace("```", ""))

# ------------------ apply_labels ------------------
apply_labels:
  - tool: github_api
    arguments:
      method: POST    
      url: $ f"https://api.github.com/repos/{steps[0].input.repo_owner}/{steps[0].input.repo_name}/issues/{steps[0].input.issue_number}/labels"
      json:
        labels: $ steps[0].input.labels

main:
  - workflow: fetch_issues
    arguments:
      repo_owner: $ steps[0].input.repo_owner
      repo_name:  $ steps[0].input.repo_name
      max_issues: $ steps[0].input.max_issues

  - workflow: fetch_labels
    arguments:
      repo_owner: $ steps[0].input.repo_owner
      repo_name:  $ steps[0].input.repo_name

  - over: $ steps[0].output.issues
    parallelism: 5
    map:
      workflow: summarize_issue
      arguments:
        issue: $ _

  - over: $ steps[2].output
    parallelism: 5
    map:
      workflow: classify_issue
      arguments:
        summary: $ _
        all_labels: $ steps[1].output.all_labels
  
  - evaluate:
      list_of_labels: $ _
      list_of_issues: $ steps[0].output.issues
    
  - evaluate:
      classified_issues: $ list(zip(_.list_of_labels, _.list_of_issues))
  
  - over: $ _.classified_issues
    parallelism: 5
    map:
      workflow: apply_labels
      arguments:
        repo_owner:    $ steps[0].input.repo_owner
        repo_name:     $ steps[0].input.repo_name
        issue_number:  $ _[1].number
        labels:        $ _[0].labels
'''


task_def=yaml.safe_load(task_yaml)

In [29]:
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

Creating an `Execution`

In [30]:
execution = client.executions.create(
    task_id=task.id,
    input={
        "repo_owner": "anasalatasiuni",
        "repo_name": "Julep-Github-Issue-Tagger",
        "max_issues": 2
    }
    
)


Listing `Transitions`

In [32]:
transitions = []
of = 0

while new_transitions:= client.executions.transitions.list(execution.id, limit=100, offset=of).items:
    transitions.extend(new_transitions)
    of += 100

for i, transition in enumerate(transitions):
    print(f"Transition index: {len(transitions) - i}, type: {transition.type}, current: {transition.current}, next: {transition.next}")
    print(transition.output)
    print("-" * 50)

Transition index: 48, type: finish, current: Current(scope_id='06827428-3796-7fd3-8000-c78bb0ff7233', step=6, workflow='main'), next: None
[{'json': [{'id': 8600760509, 'url': 'https://api.github.com/repos/anasalatasiuni/Julep-Github-Issue-Tagger/labels/enhancement', 'name': 'enhancement', 'color': 'a2eeef', 'default': True, 'node_id': 'LA_kwDOOoq64c8AAAACAKUwvQ', 'description': 'New feature or request'}], 'content': 'W3siaWQiOjg2MDA3NjA1MDksIm5vZGVfaWQiOiJMQV9rd0RPT29xNjRjOEFBQUFDQUtVd3ZRIiwidXJsIjoiaHR0cHM6Ly9hcGkuZ2l0aHViLmNvbS9yZXBvcy9hbmFzYWxhdGFzaXVuaS9KdWxlcC1HaXRodWItSXNzdWUtVGFnZ2VyL2xhYmVscy9lbmhhbmNlbWVudCIsIm5hbWUiOiJlbmhhbmNlbWVudCIsImNvbG9yIjoiYTJlZWVmIiwiZGVmYXVsdCI6dHJ1ZSwiZGVzY3JpcHRpb24iOiJOZXcgZmVhdHVyZSBvciByZXF1ZXN0In1d', 'headers': {'date': 'Fri, 16 May 2025 13:50:07 GMT', 'etag': 'W/"8c29fa464027c93c9d4b82032e489ff563d61cc1c89cff3eeb5dac7f03db34ff"', 'vary': 'Accept, Authorization, Cookie, X-GitHub-OTP,Accept-Encoding, Accept, X-Requested-With', 'server': 'github